In [1]:
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import scipy as sp
from scipy.sparse import lil_matrix
corpus_file="data/corpus.json"
WINDOW_SIZE=2
VOCAB_SIZE=200000
MIN_COUNT=4
k=200

In [2]:
with open(corpus_file) as f:
    corpus = json.load(f)    

In [3]:
words = set()
for sentence in tqdm(corpus):
    cur_words = sentence.split(" ")
    for word in cur_words:
        words.add(word)

  0%|          | 0/14432994 [00:00<?, ?it/s]

In [4]:
words =list(words)
words =[i for i in words if len(i)!=0]

In [5]:
count_word ={word:0 for word in words}

In [6]:
for sentence in tqdm(corpus):
    cur_words = sentence.split(" ")
    for word in cur_words:
        try:
            count_word[word]+=1
        except:
            pass

  0%|          | 0/14432994 [00:00<?, ?it/s]

In [7]:
sorted_words = [pair[0] for pair in sorted(count_word.items(), key=lambda item: item[1],reverse=True)]
sorted_words = sorted_words[:VOCAB_SIZE]
sorted_words = [word for word in sorted_words if count_word[word]>=MIN_COUNT]
words = sorted_words
del sorted_words

In [9]:
len(words)

179740

In [8]:
M = lil_matrix((len(words), len(words)), dtype=np.int32)
word2Ind = {w:idx for idx, w in enumerate(words)}
for sentence in tqdm(corpus):
    sentence = sentence.split(" ")
    for idx , word in enumerate(sentence):
        for i in range(idx+1, min(idx+WINDOW_SIZE+1, len(sentence))):
            try:
                M[word2Ind[word], word2Ind[sentence[i]]] += 1
                M[word2Ind[sentence[i]], word2Ind[word]] += 1
            except:
                pass

  0%|          | 0/14432994 [00:00<?, ?it/s]

In [9]:
with open("./data/svd_vocab.json","w") as f:
    json.dump(words,f,indent=4)

In [10]:
del corpus

In [11]:
from numpy import array
from scipy.sparse.linalg import svds
from numpy import diag
from numpy import dot

In [12]:
U, s, VT = svds(M.asfptype(),k=k)

In [13]:
S = diag(s)

In [14]:
W=U.dot(S)
Wt= np.linalg.norm(W, axis=1)
W = W / Wt[:, np.newaxis]

In [15]:
model={}
for word in words:
    model[word]=W[word2Ind[word]].tolist()

In [16]:
with open("./data/model.json","w") as f:
    json.dump(model,f,indent=4)